#### this script demostrates the use of single segment mode (or individual segment mode)
##### it will create a new satellite and raise apogee to 10000 km using small maneuvers at perigee using single segment mode. This is similar to the auto-sequence tutorial. The script assumes a scenario is open and does not contain a satellite named SingleSegmentSat
##### author: jens ramrath, agi
##### date: 23 oct 2024

### connect to running instance of STK

In [ ]:
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *
from agi.stk12.stkobjects.astrogator import *

stk = STKDesktop.AttachToApplication()
root = stk.Root
sc = root.CurrentScenario

print('connected to ' + sc.InstanceName)

### create new satellite and set up MCS
we'll have
1. Initial State - keep the default
2. Propagate - replace Duration stopping condition with Periapsis
3. Maneuver - 20 m/sec intrack

In [ ]:
# create new satellite
sat = sc.Children.New(AgESTKObjectType.eSatellite, 'SingleSegmentSat')
sat.SetPropagatorType(AgEVePropagatorType.ePropagatorAstrogator)
prop = sat.Propagator

# initial state
# get handle to initial state
iSegment = prop.MainSequence.Item(0)

# propagate
# get handle to propagate segment
pSegment = prop.MainSequence.Item(1)
# add periapsis stopping condition
pSegment.StoppingConditions.Add('Periapsis')
# remove default Duration stopping condition
pSegment.StoppingConditions.Remove('Duration')
# add apoapsis radius result so we can rasily query it
pSegment.Results.Add('Radius of Apoapsis')

# maneuver
# add maneuver at end
mSegment = prop.MainSequence.InsertByName('Maneuver', '-')
# specify small thrust
mSegment.Maneuver.AttitudeControl.DeltaVMagnitude = 0.02

### RUN

In [ ]:
# start new run
prop.BeginRun()

# add initial state
iSegment.Run()

# add propagate segment
pSegment.Run()

# propagate
prop.EndRun()

# get final apo radius
rop = pSegment.GetResultValue('Radius of Apoapsis')
print('apogee radius: ' + str(rop))

# add maneuver and propagate segments until perigee radius is above limit
while rop < 10000:
    # last segment state
    rSegment = prop.MainSequence.GetItemByName('-')
    
    # append to last segment
    prop.AppendRun()

    # add maneuver and propagate
    mSegment.Run()
    pSegment.Run()

    # propagate
    prop.EndRun()
    
    # get apo radius
    rop = pSegment.GetResultValue('Radius of Apoapsis')
    print('apogee radius: ' + str(rop))